In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    NoSuchElementException,
    TimeoutException,
    StaleElementReferenceException,
    WebDriverException,
    ElementClickInterceptedException,
)
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import random

# Your ZIP code string (replace with your actual full string)
zip_string = (
    "1500115003150041500515006150071500915010150121501415015150171501815019150201502115022150241502515026150271502815030150311503315034150351503715038150421504315044150451504615047150491505015051150521505315054150551505615057150591506015061150621506315064150651506615067150681507115072150741507515076150771507815081150821508315084150851508615087150881508915090151011510215104151061510815110151121511615120151221512615129151311513215133151351513615137151391514015142151431514415145151461514715148152011520215203152041520515206152071520815209152101521115212152131521415215152161521715218152191522015221152221522315224152251522615227152281522915232152331523415235152361523715238152391524115243152601527515282152901530115310153111531215313153141531515316153171532015321153221532315324153251532715329153301533115332153331533415337153381534015341153421534415345153461534715348153491535015351153521535315357153581535915360153611536215363153641536515366153671536815370153761537715378153791538015401154101541115412154131541615417154191542015421154221542315424154251542715428154291543015431154321543315434154351543615437154381544015442154431544415445154461544715448154491545015451154541545515456154581545915460154611546215463154641546515466154671546815469154701547215473154741547515476154771547815479154801548215483154841548615488154891549015492155011550215510155201552115522155301553115532155331553415535155361553715538155391554015541155421554415545155461554715550155511555215554155551555715558155591556015561155621556315564155651560115610156111561215613156151561615617156181561915620156221562315624156251562615627156281562915631156321563315634156351563615637156381563915640156411564215644156461564715650156551565615658156601566115662156631566515666156681567015671156721567315674156751567615677156781567915680156811568215683156841568615687156881568915690156911569215693156951569615697156981570115705157101571115712157131571415715157161571715720157211572215723157241572515727157281572915730157311573215733157341573615737157381573915741157421574415745157461574715748157501575215753157541575615757157591576015761157621576315764157651576715770157711577215773157741577515776157771577815779157801578115783157841580115821158231582415825158271582815829158311583215834158401584115845158461584715848158491585115853158561585715860158611586315864158651586615868158701590115902159041590515906159091592015921159221592315924159251592615927159281592915930159311593415935159361593715938159401594215943159441594515946159481594915951159521595315954159551595615957159581596015961159621596316001160021602016022160231602416025160271602816029160301603316034160351603616037160381604016041160451604616048160491605016051160521605316054160551605616057160591606116063160661610116102161051611016111161121611316114161151611616117161201612116123161241612516127161301613116132161331613416136161371614016141161421614316145161461614816150161511615316154161551615616157161591616016161161721620116210162111621216213162141621716218162201622216223162241622616228162291623016232162331623516236162381623916240162421624416245162461624816249162501625316254162551625616258162591626016262162631630116311163121631316314163161631716319163211632216323163261632716328163291633116332163331633416335163401634116342163431634416345163461634716350163511635216353163541636016361163621636416365163701637116372163731637416401164021640316404164051640616407164101641116412164151641616417164201642116422164231642416426164271642816430164331643416435164361643816440164411644216443164441647516501165021650316504165051650616507165081650916510165111654616550165631660116602166111661316616166171661916620166211662216623166241662516627166301663116633166341663516636166371663816639166401664116645166461664716648166501665116652166541665516656166571665916661166621666416665166661666716668166691667016671166721667316674166751667716678166791668016682166831668416685166861668916691166921669316694166951669916701167201672416725167261672716729167301673116732167331673416735167381674016743167441674516746167481674916750168011680216803168201682116822168231682516826168271682816829168301683216833168341683516836168371683816839168401684116843168441684516847168481684916851168521685316854168551685616858168591686016861168631686416865168661686816870168711687216874168751687616877168781687916881168821690116910169111691216914169151691716920169211692216923169251692616927169281692916930169321693316935169361693716938169391694016941169421694316946169471694816950170021700317004170051700617007170091701017011170131701417015170161701717018170191702017021170221702317024170251702617027170281702917030170321703317034170351703617037170381703917040170411704217043170441704517046170471704817049170501705117052170531705417055170561705717058170591706017061170621706317064170651706617067170681706917070170711707317074170751707617077170781708017081170821708417086170871708817090170931709417097170981709917101171021710317104171091711017111171121711317120172011720217210172111721217213172141721517217172191722017221172221722317224172251722817229172321723317235172361723717238172391724017241172431724417246172471724917250172511725217253172541725517257172601726117262172631726417265172661726717268172711730117302173031730417306173071730917311173131731417315173161731717318173191732017321173221732317324173251732717329173311733917340173421734317344173451734717349173501735217353173551735617360173611736217363173641736517366173681737017371173721740117402174031740417406174071740817501175021750417505175061750717508175091751217516175171751817519175201752217527175291753217534175351753617538175401754317545175471755017551175521755417555175571756017562175631756517566175681756917570175721757617578175791758117582175841760117602176031760617701177021772017721177231772417727177281772917730177311773717739177401774217744177451774717748177491775017751177521775417756177581776017762177631776417765177681776917771177721777417776177771777817779178011781017812178131781417815178201782117822178231782417827178291783017832178341783517836178371784017841178421784417845178461784717850178511785317855178561785717858178591786017861178621786417865178661786717868178701787217876178781788017881178841788517886178871788817889179011792017921179221792317925179291793017931179331793417935179361793817941179431794417945179461794817949179511795217953179541795717959179601796117963179641796517967179681797017972179741797617978179791798017981179821798317985180111801218013180141801518017180181802018030180311803218034180351803618037180381804018041180421804518046180491805118052180531805418055180561805818059180621806318064180651806618067180691807018071180721807318074180761807718078180791808018081180831808518086180871808818091180921810118102181031810418106181091819518201182021821018211182121821418216182181821918220182211822218223182241822518229182301823118232182341823518237182391824018241182421824418245182461824818249182501825218254182551825618301183021832118322183231832418325183261832718328183301833118332183331833418335183361833718340183421834318344183461834718349183501835118353183541835518356183571836018370183711837218403184051840718411184131841418415184171841918420184211842418425184261842818430184311843318434184351843618437184381843918440184411844318444184451844618447184511845218453184541845518456184571845818460184611846218463184641846518466184691847018471184721847318503185041850518507185081850918510185121851718518185191860218603186101861218614186151861618617186181861918621186221862318624186251862618627186281862918630186311863218634186351863618640186411864218643186441865118655186561865718660186611870118702187041870518706187071870818709188011881018812188141881618817188181882018821188221882318824188251882618828188291883018831188321883318834188371884018842188431884418845188461884718848188501885118853188541890118902189121891318914189151891718920189211892318925189271892918930189321893318935189361893818940189421894418946189471895018951189541895518960189621896418966189691897018972189741897618977189801900119002190031900419006190071900819009190101901219013190141901519017190181902019021190221902319025190261902719029190301903119032190331903419035190361903819040190411904319044190461904719050190521905319054190551905619057190601906119063190641906619067190701907219073190741907519076190781907919081190821908319085190861908719090190941909519096191021910319104191061910719108191091911119112191131911419115191161911819119191201912119122191231912419125191261912719128191291913019131191321913319134191351913619137191381913919140191411914219143191441914519146191471914819149191501915119152191531915419301193101931119312193161931719319193201933019333193351934119342193431934419345193481935019352193551935819362193631936519367193691937219373193741937519380193821938319390194011940319405194061942219425194261942819435194361943719438194401944219444194461945319454194561945719460194621946419465194681947219473194741947519477194901949219501195031950419505195061950719508195101951119512195161951819520195221952319525195261952919530195331953419535195361953819539195401954119543195441954519547195491955019551195541955519559195601956219564195651956719601196021960419605196061960719608196091961019611"  # truncated for example — replace with full string
    # ...
)

# Parse ZIP codes
zip_list = [zip_string[i : i + 5] for i in range(0, len(zip_string), 5)]
zip_list = [z for z in zip_list if len(z) == 5 and z.isdigit()]

url = "https://www.dtnpf.com/agriculture/web/ag/markets/local-grain-bids"

# Chrome options to disable images, extensions and run headless
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
prefs = {
    "profile.managed_default_content_settings.images": 2,
    "profile.managed_default_content_settings.stylesheets": 2,
    "profile.default_content_setting_values.notifications": 2,
    # Keep JavaScript enabled so the site works
    "profile.managed_default_content_settings.javascript": 1,
}
chrome_options.experimental_options["prefs"] = prefs


def create_driver():
    driver = webdriver.Chrome(options=chrome_options)
    driver.set_page_load_timeout(20)
    return driver


def close_cookie_banner(driver):
    try:
        wait = WebDriverWait(driver, 5)
        accept_button = wait.until(
            EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))
        )
        accept_button.click()
        WebDriverWait(driver, 3).until(
            EC.invisibility_of_element_located((By.ID, "onetrust-button-group-parent"))
        )
        print("[Info] Cookie banner accepted and closed.")
    except (TimeoutException, NoSuchElementException):
        # No cookie banner present
        pass


def click_with_retry(driver, element, retries=3):
    for attempt in range(retries):
        try:
            driver.execute_script("arguments[0].scrollIntoView(true);", element)
            WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".js-simpleLocationView-submit")))
            driver.execute_script("arguments[0].click();", element)
            return True
        except ElementClickInterceptedException:
            print(f"[Warning] Click intercepted, retry {attempt + 1}/{retries}")
            time.sleep(1)
    return False


def scrape_zip(zip_code, max_retries=3):
    local_results = []
    for attempt in range(1, max_retries + 1):
        driver = None
        try:
            driver = create_driver()
            driver.get(url)
            wait = WebDriverWait(driver, 15)

            close_cookie_banner(driver)

            wait.until(EC.presence_of_element_located((By.NAME, "postalCode")))
            zip_input = driver.find_element(By.NAME, "postalCode")
            zip_input.clear()
            zip_input.send_keys(zip_code)

            go_button = driver.find_element(By.CSS_SELECTOR, ".js-simpleLocationView-submit")

            if not click_with_retry(driver, go_button):
                print(f"[Error] Could not click submit button for ZIP: {zip_code}")
                return []

            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.box")))

            boxes = driver.find_elements(By.CSS_SELECTOR, "div.box")
            for i in range(len(boxes)):
                try:
                    boxes = driver.find_elements(By.CSS_SELECTOR, "div.box")  # re-find to avoid stale element
                    box = boxes[i]

                    try:
                        header = box.find_element(By.CSS_SELECTOR, "div.box-hd h4").text.strip()
                    except NoSuchElementException:
                        header = None

                    rows = box.find_elements(By.CSS_SELECTOR, "tbody tr")
                    for j in range(len(rows)):
                        try:
                            rows = box.find_elements(By.CSS_SELECTOR, "tbody tr")  # re-find rows
                            row = rows[j]

                            cells = row.find_elements(By.TAG_NAME, "td")
                            if len(cells) >= 4:
                                local_results.append(
                                    {
                                        "zip": zip_code,
                                        "location": header,
                                        "commodity": cells[0].text.strip(),
                                        "price": cells[1].text.strip(),
                                        "basis": cells[2].text.strip(),
                                        "date": cells[3].text.strip(),
                                    }
                                )
                        except StaleElementReferenceException:
                            continue

                except StaleElementReferenceException:
                    continue

            return local_results

        except TimeoutException:
            print(f"[Timeout] Attempt {attempt} for ZIP: {zip_code}")
        except WebDriverException as e:
            print(f"[WebDriverError] Attempt {attempt} for ZIP: {zip_code} - {e}")
        except Exception as e:
            print(f"[Error] Attempt {attempt} for ZIP: {zip_code} - {e}")

        finally:
            if driver:
                driver.quit()

        time.sleep(random.uniform(2, 5))

    print(f"[Skipped] ZIP: {zip_code} after {max_retries} attempts")
    return []


def main():
    max_workers = 4  # Adjust to your CPU/network constraints
    all_results = []

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(scrape_zip, zip_code): zip_code for zip_code in zip_list}

        for future in as_completed(futures):
            zip_code = futures[future]
            try:
                data = future.result()
                if data:
                    all_results.extend(data)
            except Exception as exc:
                print(f"[Error] ZIP {zip_code} generated an exception: {exc}")

    # Create pandas DataFrame for further use in notebook
    global df
    df = pd.DataFrame(all_results)
    print(f"Scraping completed. {len(df)} records collected.")


if __name__ == "__main__":
    main()


[Info] Cookie banner accepted and closed.
[Info] Cookie banner accepted and closed.
[Info] Cookie banner accepted and closed.
[Info] Cookie banner accepted and closed.
[Timeout] Attempt 1 for ZIP: 15010
[Info] Cookie banner accepted and closed.
[Info] Cookie banner accepted and closed.
[Info] Cookie banner accepted and closed.
[Info] Cookie banner accepted and closed.
[Timeout] Attempt 1 for ZIP: 15012
[Info] Cookie banner accepted and closed.
[Info] Cookie banner accepted and closed.
[Info] Cookie banner accepted and closed.
[Info] Cookie banner accepted and closed.
[Info] Cookie banner accepted and closed.
[Info] Cookie banner accepted and closed.
[Info] Cookie banner accepted and closed.
[Info] Cookie banner accepted and closed.
[Info] Cookie banner accepted and closed.
[Info] Cookie banner accepted and closed.
[Info] Cookie banner accepted and closed.
[Info] Cookie banner accepted and closed.
[Info] Cookie banner accepted and closed.
[Info] Cookie banner accepted and closed.
[Info]

In [2]:
df_unique = df.drop_duplicates(subset=[col for col in df.columns if col != 'zip'])
df_unique

,zip,location,commodity,price,basis,date
0,15005,"HERITAGE COOPERATIVE - EAST LIVERPOOL, OH",Soybeans,$9.56,-0.40,8/8/2025
1,15005,"HERITAGE COOPERATIVE - EAST LIVERPOOL, OH",Corn,$3.64,-0.20,8/8/2025
2,15005,"HERITAGE COOPERATIVE - EAST LIVERPOOL, OH",Corn,$3.39,-0.45,8/8/2025
3,15005,"HERITAGE COOPERATIVE - EAST LIVERPOOL, OH",Soybeans,$9.21,-0.75,8/8/2025
5,15005,"AG CENTRAL - NEW CASTLE, PA",Corn,$3.64,-0.20,8/8/2025
...,...,...,...,...,...,...
14690,19001,"PERDUE FARMS - MARIETTA, PA",Soybeans,$10.23,0.25,8/8/2025
16065,19362,"PERDUE FARMS - MARIETTA, PA",Soybeans,$9.67,-0.11 (est.),8/8/2025
16535,19506,"TRIPLE M FARMS - LEBANON, PA",Soybeans,$10.21,0.25,8/8/2025
16544,19507,"TRIPLE M FARMS - LEBANON, PA",Soybeans,$9.89,0.11 (est.),8/8/2025


In [5]:
import os

# Define path to Desktop
desktop_path = os.path.expanduser("~/Desktop/grain_bids.csv")

# Save DataFrame without the index
df_unique.to_csv(desktop_path, index=False)

print(f"File saved to: {desktop_path}")


File saved to: /Users/nshaffer/Desktop/grain_bids.csv


In [11]:
import pandas as pd

# Show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


df_unique

,zip,location,commodity,price,basis,date
0,15005,"HERITAGE COOPERATIVE - EAST LIVERPOOL, OH",Soybeans,$9.56,-0.40,8/8/2025
1,15005,"HERITAGE COOPERATIVE - EAST LIVERPOOL, OH",Corn,$3.64,-0.20,8/8/2025
2,15005,"HERITAGE COOPERATIVE - EAST LIVERPOOL, OH",Corn,$3.39,-0.45,8/8/2025
3,15005,"HERITAGE COOPERATIVE - EAST LIVERPOOL, OH",Soybeans,$9.21,-0.75,8/8/2025
5,15005,"AG CENTRAL - NEW CASTLE, PA",Corn,$3.64,-0.20,8/8/2025
6,15005,CENTERRA COOP-GENE COOPER FARMS - SLIPPERY ROC...,Corn,$3.39,-0.45,8/8/2025
7,15005,CENTERRA COOP-GENE COOPER FARMS - SLIPPERY ROC...,Soybeans,$9.21,-0.75,8/8/2025
21,15004,"HERITAGE COOPERATIVE - LISBON, OH",Corn,$3.64,-0.20,8/8/2025
22,15004,"HERITAGE COOPERATIVE - LISBON, OH",Soybeans,$9.56,-0.40,8/8/2025
34,15006,CENTERRA COOP-GENE COOPER FARMS - SLIPPERY ROC...,Corn,$3.64,-0.20,8/8/2025
